In [63]:
#load libraries
require(tidyr)
require(dplyr)

In [64]:
#load Colony_Data.csv
colony=read.csv("~/Documents/SCTLD/SCTLD_samples/Sample_Data/CBC_ColonyData.csv")

In [65]:
#isolate immune samples from colony data
colony_immune=colony[colony$immune_y.n=="y",]
colony_immune=as.vector(paste0(colony_immune$TransectNum,"_", colony_immune$NewTagNum))

In [66]:
#load metagenomics data
metagenomics=read.csv("~/Documents/SCTLD/SCTLD_samples/Sample_Data/Metagenomics_Tracker_Belize.csv")

In [67]:
#waiting for a pull request review
#add the extraction location to 012024 samples
metagenomics <- metagenomics %>%
  mutate(Extraction_physical_location = case_when(
    (Extraction_physical_location == "" | is.na(Extraction_physical_location)) & 
    Date_Extracted %in% c("2_17_2025", "2_20_2025") ~ "UML_NARWHAL_R2_B16",
    TRUE ~ Extraction_physical_location))

In [68]:
#using an old copy of sample data from github bc i messed around with it and need to wait for pull review
sample=read.csv("~/Downloads/CBC_samples1.csv")

# sample_dna add colony column to sample data
sample$colony= colony= c(paste0(sample$TransectNum, "_", sample$NewTagNum))

#clean sample data of leading spaces
sample$Month_year <- trimws(sample$Month_year)

# only utilize EtOH and RNALater samples and exclue 062025 samples
sample_DNA <- sample[(sample$Sample_type == "Core_EtOH" | sample$Sample_type == "Core_RNAlater") &
    sample$Month_year != "62025",]

#double check that all 062025 samples have been removed
unique(sample_DNA$Month_year)

#view sample_dna, this is a new table modeled after sample data sheet and utilizing the colony tag num as a column
head(sample_DNA)

[1] "92023"  "122022" "52022"  "112023" "42024"  "102019" "62019"  "122023"
 [9] "12024"  "22024"  "62024"  "82024"

,X.1,X,Month_year,Country,Location,CollectionDate,Transect,TransectNum,OldTagNum,NewTagNum,⋯,Sample_type,SampleNum,Health_status,Sampling_notes,Tubelabel_species,Sample_physical_location,Extraction_physical_location,Date_sequenced,Notes,colony
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,2,2,92023,BEL,CBC,9/27/23,CBC30N,1,,1,⋯,Core_RNAlater,185,Diseased_Margin,only margin sample available,092023_BEL_CBC_T1_185_SSID,UML_NARWHAL_R1_B10,,,,1_1
3,3,3,92023,BEL,CBC,9/25/23,CBC30N,1,,2,⋯,Core_RNAlater,171,Healthy,CLP 90%,092023_BEL_CBC_T1_171_PAST,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,,,1_2
4,4,4,92023,BEL,CBC,9/25/23,CBC30N,1,,3,⋯,Core_RNAlater,173,Healthy,CLP 80%; DC 20%,092023_BEL_CBC_T1_173_SSID,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,,,1_3
5,5,5,92023,BEL,CBC,9/25/23,CBC30N,1,,12,⋯,Core_RNAlater,177,Healthy,No CL,092023_BEL_CBC_T1_177_PSTR,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,,,1_12
6,6,6,92023,BEL,CBC,9/25/23,CBC30N,1,,13,⋯,Core_RNAlater,175,Healthy,No CL,092023_BEL_CBC_T1_175_PAST,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,,R2_B15 EXTRACTED TWICE,1_13
7,7,7,92023,BEL,CBC,9/27/23,CBC30N,1,,5,⋯,Core_RNAlater,188,Diseased_Tissue,,092023_BEL_CBC_T1_188_SSID,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B11,,,1_5


In [69]:
#changing X column to Tubelabel_species
names(metagenomics)[names(metagenomics) == 'X'] <- 'Tubelabel_species'

In [70]:
colnames(metagenomics)

[1] "Tubelabel_species"                                                  
 [2] "Health_Status"                                                      
 [3] "Starting_Weight"                                                    
 [4] "Date_Extracted"                                                     
 [5] "Raw_ng_ul"                                                          
 [6] "Date_Enriched"                                                      
 [7] "Microbe_ng_ul"                                                      
 [8] "Microbe_Location"                                                   
 [9] "Microbe_clean_date.n"                                               
[10] "Host_ng_ul"                                                         
[11] "Host_Location"                                                      
[12] "Host_clean_y.n"                                                     
[13] "Date_Libprep"                                                       
[14] "Status"                                                             
[15] "Notes"                                                              
[16] "Seq_date"                                                           
[17] "Host_Seq_date"                                                      
[18] "Microbe_seq_file"                                                   
[19] "Host_seq_file"                                                      
[20] "Seq_Location..in.Unity."                                            
[21] "Sample_physical_location"                                           
[22] "Extraction_physical_location"                                       
[23] "Location_notes"                                                     
[24] "Sample_Code...datecollected_tagnumber_transect_samplenumber_species"

In [71]:
#the columns that I want to isolate from metagenomics tracker
metagenomics_PCR = metagenomics[c("Tubelabel_species", "Health_Status", "Date_Extracted", "Raw_ng_ul", "Extraction_physical_location", "Date_Enriched","Microbe_Location")]

In [72]:
head(metagenomics_PCR)

,Tubelabel_species,Health_Status,Date_Extracted,Raw_ng_ul,Extraction_physical_location,Date_Enriched,Microbe_Location
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,052022_BEL_CBC_T2_45_PSTR,Diseased_Margin,10_10_2023,22.8,UML_NARWHAL_R2_B3,10_17_2023,UML_NARWHAL_R2_B1
2,052022_BEL_CBC_T2_46_PSTR,Diseased_Tissue,10_10_2023,63.9,UML_NARWHAL_R2_B3,10_17_2023,UML_NARWHAL_R2_B1
3,052022_BEL_CBC_T2_59_OFAV,Healthy,10_3_2023,73.7,UML_NARWHAL_R2_B3,10_16_2023,UML_NARWHAL_R2_B1
4,042024_BEL_CBC_T1_925_PAST,Healthy,8_19_2024,5.46,UML_NARWHAL_R2_B29,9_28_2024,UML_NARWHAL_R6_B30
5,052022_BEL_CBC_T2_72_OFAV,Healthy,10_3_2023,47.6,UML_NARWHAL_R2_B3,10_11_23,UML_NARWHAL_R6_B1
6,052022_BEL_CBC_T2_71_OFAV,Healthy,10_3_2023,98.8,UML_NARWHAL_R2_B3,10_16_2023,UML_NARWHAL_R6_B1


In [73]:
#info I am using to create ties between sample_dna and metagenomics_PCR
sample_DNA=sample_DNA[c("Tubelabel_species", "colony")]

In [74]:
#only including immune samples in sample_DNA
immune_sample_DNA=sample_DNA[sample_DNA$colony %in% colony_immune,]

In [75]:
head(immune_sample_DNA)

,Tubelabel_species,colony
,<chr>,<chr>
3,092023_BEL_CBC_T1_171_PAST,1_2
4,092023_BEL_CBC_T1_173_SSID,1_3
5,092023_BEL_CBC_T1_177_PSTR,1_12
6,092023_BEL_CBC_T1_175_PAST,1_13
12,092023_BEL_CBC_T1_172_PAST,1_21
13,092023_BEL_CBC_T1_174_MCAV,1_24


In [76]:
nrow(metagenomics_PCR)

[1] 557

In [77]:
nrow(immune_sample_DNA)

[1] 368

In [78]:
#check to see why there is a discrepancy from immune_sample_DNA to_metagenomics_PCR
unmatched=anti_join(immune_sample_DNA,metagenomics_PCR, by='Tubelabel_species')
nrow(unmatched)

[1] 35

In [79]:
# Identify the incorrect labels that contain "Belize"
incorrect_names <- immune_sample_DNA$Tubelabel_species[grepl("Belize", immune_sample_DNA$Tubelabel_species)]
print(incorrect_names)

 [1] "022024_Belize_CBC_T1_771_PAST" "022024_Belize_CBC_T1_773_SSID"
 [3] "022024_Belize_CBC_T1_776_PSTR" "022024_Belize_CBC_T1_774_MCAV"
 [5] "022024_Belize_CBC_T2_810_SSID" "022024_Belize_CBC_T2_813_MCAV"
 [7] "022024_Belize_CBC_T2_812_PAST" "022024_Belize_CBC_T2_814_MCAV"
 [9] "022024_Belize_CBC_T2_811_PAST" "022024_Belize_CBC_T2_808_PAST"
[11] "022024_Belize_CBC_T2_809_MCAV" "022024_Belize_CBC_T2_805_SSID"
[13] "022024_Belize_CBC_T2_806_SSID" "022024_Belize_CBC_T2_807_SSID"
[15] "022024_Belize_CBC_T2_804_OFAV" "022024_Belize_CBC_T2_815_OFAV"
[17] "022024_Belize_CBC_T3_842_MCAV" "022024_Belize_CBC_T3_843_MCAV"
[19] "022024_Belize_CBC_T3_845_SSID" "022024_Belize_CBC_T3_847_PAST"
[21] "022024_Belize_CBC_T3_848_MCAV" "022024_Belize_CBC_T3_849_PSTR"
[23] "022024_Belize_CBC_T3_850_MCAV" "022024_Belize_CBC_T3_853_SSID"
[25] "022024_Belize_CBC_T3_854_PAST" "022024_Belize_CBC_T3_844_OFAV"
[27] "022024_Belize_CBC_T3_852_OFAV" "022024_Belize_CBC_T3_851_PSTR"
[29] "022024_Belize_CBC_T4_870_MCA

In [80]:
# Replace "Belize" with "BEL" for the problematic names
immune_sample_DNA$Tubelabel_species <- gsub("Belize", "BEL", immune_sample_DNA$Tubelabel_species)

In [81]:
# Check to ensure only "Belize" was replaced with "BEL"
print(immune_sample_DNA$Tubelabel_species[grepl("BEL", immune_sample_DNA$Tubelabel_species)])

  [1] "092023_BEL_CBC_T1_171_PAST"      "092023_BEL_CBC_T1_173_SSID"     
  [3] "092023_BEL_CBC_T1_177_PSTR"      "092023_BEL_CBC_T1_175_PAST"     
  [5] "092023_BEL_CBC_T1_172_PAST"      "092023_BEL_CBC_T1_174_MCAV"     
  [7] "092023_BEL_CBC_T1_178_OANN"      "092023_BEL_CBC_T2_183_MCAV"     
  [9] "092023_BEL_CBC_T2_182_PAST"      "092023_BEL_CBC_T2_184_MCAV"     
 [11] "092023_BEL_CBC_T2_181_PAST"      "092023_BEL_CBC_T2_175_PAST"     
 [13] "092023_BEL_CBC_T2_176_MCAV"      "092023_BEL_CBC_T2_174_SSID"     
 [15] "092023_BEL_CBC_T2_173_SSID"      "092023_BEL_CBC_T2_172_OFAV"     
 [17] "092023_BEL_CBC_T2_171_OFAV"      "092023_BEL_CBC_T2_179_SSID"     
 [19] "092023_BEL_CBC_T3_176_MCAV"      "092023_BEL_CBC_T3_177_SSID"     
 [21] "092023_BEL_CBC_T3_178_PAST"      "092023_BEL_CBC_T3_181_PAST"     
 [23] "092023_BEL_CBC_T3_175_MCAV"      "092023_BEL_CBC_T3_172_PSTR"     
 [25] "092023_BEL_CBC_T3_182_MCAV"      "092023_BEL_CBC_T3_183_SSID"     
 [27] "092023_BEL_CBC_T3_174_OFAV"    

In [82]:
# inner join 
immune_metagenomics_PCR <- merge(x=metagenomics_PCR,y=immune_sample_DNA, 
                by="Tubelabel_species")
head(immune_metagenomics_PCR)

,Tubelabel_species,Health_Status,Date_Extracted,Raw_ng_ul,Extraction_physical_location,Date_Enriched,Microbe_Location,colony
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,012024_BEL_CBC_T1_557_SSID,Healthy,2_17_2025,44.6,UML_NARWHAL_R2_B16,,,1_3
2,012024_BEL_CBC_T1_559_MCAV,Healthy,2_17_2025,14.8,UML_NARWHAL_R2_B16,,,1_24
3,012024_BEL_CBC_T1_561_OANN,Healthy,2_17_2025,18,UML_NARWHAL_R2_B16,,,1_25
4,012024_BEL_CBC_T1_563_PSTR,Healthy,2_17_2025,35.8,UML_NARWHAL_R2_B16,,,1_12
5,012024_BEL_CBC_T1_565_PAST,Healthy,2_17_2025,7.72,UML_NARWHAL_R2_B16,,,1_21
6,012024_BEL_CBC_T2_585_OFAV,Healthy,2_17_2025,71.6,UML_NARWHAL_R2_B16,,,2_76


In [83]:
length(unique(immune_metagenomics_PCR$colony))

[1] 37

In [84]:
nrow(immune_sample_DNA)

[1] 368

In [85]:
nrow(immune_metagenomics_PCR)

[1] 373

In [86]:
#check to see why there is a discrepancy from immune_sample_DNA to immune_metagenomics_PCR
immune_unmatched=anti_join(immune_sample_DNA, immune_metagenomics_PCR, by='Tubelabel_species')
nrow(immune_unmatched)
(immune_unmatched)

[1] 0

Tubelabel_species,colony
<chr>,<chr>


In [87]:
#manually corrected for these unmatched values in metagenomics tracker and CBC_samples
#mostly my own typos
#find 052022_BEL_CBC_T3_25_PAST extraction
#do we want to extract 012024?

In [88]:
#make spreadsheet with all immune metagenomics 
write.csv(immune_metagenomics_PCR, file="~/Documents/SCTLD/Coral_Code/NSF_Rapid/CBC_PCR_Plan/2_immune_metagenomics_PCR.csv")

In [132]:
# Step 1: Load your data
immune_metagenomics_PCR <- read.csv("2_immune_metagenomics_PCR.csv")

# Step 2: Shuffle the rows
randomized_immune_metagenomics_PCR <- immune_metagenomics_PCR[sample(nrow(immune_metagenomics_PCR)), ]

# Step 3: Save the shuffled data to a new .csv file
write.csv(randomized_immune_metagenomics_PCR, "~/Documents/SCTLD/Coral_Code/NSF_Rapid/CBC_PCR_Plan/2_randomized_immune_metagenomics_PCR.csv", row.names = FALSE)

In [54]:



#now I want to coelese the order so the first 96 samples match randomized_immune_metagenomics_PCR
#first I'll make "Date_16S" and Date_ITS2 columns in 2_randomized

#open files 
two= read.csv("~/Documents/SCTLD/Coral_Code/NSF_Rapid/CBC_PCR_Plan/2_randomized_immune_metagenomics_PCR.csv")
one=read.csv("~/Documents/SCTLD/Coral_Code/NSF_Rapid/CBC_PCR_Plan/randomized_immune_metagenomics_PCR.csv")

# Step 1: Create 'Date_16S' column in df 'two' with NA values
two <- two %>%
  mutate(Date_16S = NA)

# Step 2: Coalesce values (e.g., combine one$Date_16S and two$Date_16S)
two$Date_16S <- coalesce(one$Date_16S[match(two$Tubelabel_species, one$Tubelabel_species)], two$Date_16S)

In [55]:
class(two$Date_16S)

[1] "character"

In [56]:
two <- two %>%
  mutate(
    # Ensure character
    Date_16S = as.character(Date_16S),

    # Replace blanks with NA
    Date_16S = ifelse(Date_16S == "", NA, Date_16S),

    # Temporarily convert to Date object
    Date_parsed = as.Date(Date_16S, format = "%m_%d_%Y")
  ) %>%
  
  # Sort using parsed Date
  arrange(is.na(Date_parsed), Date_parsed) %>%
  
  # Restore original format (m_d_Y) for display
  mutate(
    Date_16S = ifelse(
      is.na(Date_parsed),
      NA,
      format(Date_parsed, "%-m_%-d_%Y")  # e.g., 1_2_2025
    )
  ) %>%
  
  # Drop the temporary parsed date column
  select(-Date_parsed)


In [57]:
head(two$Date_16S)

[1] "1_2_2025" "1_2_2025" "1_2_2025" "1_2_2025" "1_2_2025" "1_2_2025"

In [58]:
unique(two$Date_16S)

[1] "1_2_2025"  "1_22_2025" "1_23_2025" "2_24_2025" "2_25_2025" "2_26_2025"
 [7] "2_27_2025" "3_4_2025"  "3_6_2025"  "3_7_2025"  "3_10_2025" NA

In [59]:
#repeat with ITS2
# Step 1: Create 'Date_ITS2' column in df 'two' with NA values
two <- two %>%
  mutate(Date_ITS2 = NA)

# Step 2: Coalesce values (e.g., combine one$Date_16S and two$Date_16S)
two$Date_ITS2 <- coalesce(one$Date_ITS2[match(two$Tubelabel_species, one$Tubelabel_species)], two$Date_ITS2)

In [60]:
two <- two %>%
  mutate(
    # Ensure character
    Date_ITS2 = as.character(Date_ITS2),

    # Replace blanks with NA
    Date_ITS2 = ifelse(Date_ITS2 == "", NA, Date_ITS2),

    # Temporarily convert to Date object
    Date_ITS2_parsed = as.Date(Date_ITS2, format = "%m_%d_%Y")
  ) %>%
  
  # Sort using parsed Date
  arrange(is.na(Date_ITS2_parsed), Date_ITS2_parsed) %>%
  
  # Restore original format (m_d_Y) for display
  mutate(
    Date_ITS2 = ifelse(
      is.na(Date_ITS2_parsed),
      NA,
      format(Date_ITS2_parsed, "%-m_%-d_%Y")  # e.g., 1_2_2025
    )
  ) %>%
  
  # Drop the temporary parsed date column
  select(-Date_ITS2_parsed)
unique(two$Date_ITS2)

[1] "6_26_2025" "6_27_2025" "6_30_2025" "7_2_2025"  "7_3_2025"  "7_7_2025" 
 [7] "7_9_2025"  "7_10_2025" "7_11_2025" NA

In [62]:
write.csv(two, file="~/Documents/SCTLD/Coral_Code/NSF_Rapid/CBC_PCR_Plan/2_randomized_immune_metagenomics_PCR.csv")